In [1]:
import os
from dotenv import load_dotenv
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.yfinance import YFinanceTools
from agno.tools.exa import ExaTools

In [2]:
load_dotenv()

API_BASE_URL = os.getenv("API_BASE_URL")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
EXA_API_KEY = os.getenv("EXA_API_KEY")

In [3]:
agent_a = Agent(
    model=OpenAIChat(id=OPENAI_MODEL_NAME, api_key=OPENAI_API_KEY, base_url=API_BASE_URL),
    tools=[YFinanceTools(stock_price=True, historical_prices=True, company_news=True, 
                         analyst_recommendations=True, company_info=True)],  
    show_tool_calls=True,
    name="Market Data Fetcher",
    description="Fetch the latest 7 days of OHLCV data for the provided stock symbol in Unix timestamp format.",
    instructions=[
        "Retrieve the last 7 days of OHLCV (Open, High, Low, Close, Volume) data for the provided stock symbol.",
        "Ensure the data is `dd-mm-yyyy` format using the formula `(Unix Timestamp - 1000000000) / 86400 + 1970-01-01`. Example: 1739491200000 -> 14-02-2025. Strictly ensure that the calculated date is correct.",
        "Return data in JSON format with the following structure:",
        "   - Unix Timestamp",
        "   - Open",
        "   - High",
        "   - Low",
        "   - Close",
        "   - Volume."
    ]
)

In [4]:
agent_b = Agent(
    model=OpenAIChat(id=OPENAI_MODEL_NAME, api_key=OPENAI_API_KEY, base_url=API_BASE_URL),
    name="Trade Anomaly Detector",
    description="Detect anomalies in OHLCV stock data by analyzing percentage change.",
    instructions=[
        "1. **Receive formatted OHLCV data from Agent A (Dates already in the correct `dd-mm-yyyy` format).**",
        "2. **Calculate daily percentage change using:**",
        "   - (Close_today - Close_yesterday) / Close_yesterday * 100.",
        "3. **Flag anomalies if the daily percentage change exceeds ±5%.**",
        "4. **Ensure the structured JSON output includes:**",
        "   - Date (dd-mm-yyyy)",
        "   - Open, High, Low, Close, Volume",
        "   - Percent Change",
        "   - Anomaly (YES/NO).",
        "5. **Send processed OHLCV data with anomalies to Agent C for financial news analysis.**"
    ]
)

In [5]:
agent_c = Agent(
    model=OpenAIChat(id=OPENAI_MODEL_NAME, api_key=OPENAI_API_KEY, base_url=API_BASE_URL),
    name="News Analysis Agent",
    tools=[
        YFinanceTools(company_news=True),  
        ExaTools(include_domains=[
            "cnbc.com", "reuters.com", "bloomberg.com", 
            "marketwatch.com", "cnn.com", "moneycontrol.com", 
            "investing.com", "ft.com", "finance.yahoo.com"
        ])  
    ],
    description="Fetches relevant financial news using OHLCV data from Agent B.",
    instructions=[
        "1. **Use ONLY the OHLCV table received from Agent B. Do NOT request the table separately.**",
        "2. **Assume the OHLCV data has already been transferred in JSON format.**",
        "3. **Fetch relevant financial news for the past 7 days covering all stock movements in the table, highlighting anomalies.**",
        "4. **Prioritize news impacting stock movement, including earnings, macroeconomic trends, industry shifts, and regulatory changes. Use Exa to fetch news.**",
        "5. **For each news article, provide:**",
        "   - **Headline & Source link**",
        "   - **Detailed Analysis** (Minimum 3-4 sentences covering stock impact, investor sentiment, and market reactions).",
    ]
)

In [6]:
agent_d = Agent(
    model=OpenAIChat(id=OPENAI_MODEL_NAME, api_key=OPENAI_API_KEY, base_url=API_BASE_URL),
    name="Sentiment Analysis Agent",
    description="Analyze sentiment from financial news articles related to a specific stock over the past 7 days.",
    instructions=[
        "1. **Receive financial news articles from Agent C covering the entire 7-day period.**",
        "2. **Perform sentiment analysis on the stock news using NLP techniques.**",
        "3. **Determine sentiment distribution as a percentage split (bullish vs bearish).**",
        "   - A higher bullish percentage indicates positive market sentiment.",
        "   - A higher bearish percentage indicates negative market sentiment.",
        "4. **Extract key phrases and tone from each article to justify the sentiment rating.**",
        "5. **Provide a structured sentiment summary including:**",
        "   - **Bullish sentiment percentage (e.g., 70% bullish).**",
        "   - **Bearish sentiment percentage (e.g., 30% bearish).**",
        "   - **Key phrases that justify the sentiment rating.**",
        "   - **Explain sentiment shifts in relation to OHLCV data anomalies and market trends.**",
        "6. **Pass structured sentiment insights to Market Analysis Team.**"
    ],
)

In [7]:
market_analysis_team = Agent(
    name="Market Analysis Team",
    model=OpenAIChat(id=OPENAI_MODEL_NAME, api_key=OPENAI_API_KEY, base_url=API_BASE_URL),
    team=[agent_a, agent_b, agent_c], 
    instructions=[
        "1. **Agent A must fetch the latest OHLCV data for the provided stock symbol, and pass it to Agent B.**",
        "2. **Agent B must detect anomalies in the OHLCV data and immediately pass the structured OHLCV table to Agent C.**",
        "3. Agent C must use ONLY the OHLCV table received from Agent B to fetch financial news."
        "4. **Generate a well-structured final report including:**",
        "   - **OHLCV Data Table (with anomalies and daily percentage changes) from Agent B.**",
        "   - **Comprehensive News Analysis (covering all stock movements, not just anomalies) from Agent C with the source hyperlink.**",
        "   - **Conclusion summarizing stock trends, news impacts, and investment considerations.**",
        "5. **Ensure formatting consistency across all sections before compiling the final report.**",
    ],
    show_tool_calls=True,
    markdown=True,
)

In [8]:
market_analysis_team.print_response("Analyze real-time stock anomalies and news for TSLA", debug=True, stream=True)

Output()